Chúng ta sẽ bắt đầu xây dựng chủ đề **T5: Phân tích tính giống thuốc bằng RDKit** sử dụng dữ liệu từ **ChEMBL 35** và tích hợp với PostgreSQL, theo tiêu chuẩn AIMLOps.

---

## 🎯 Mục tiêu / Objective

**VIETNAMESE**:
Phân tích "tính giống thuốc" (drug-likeness) là một bước thiết yếu trong quá trình sàng lọc phân tử trước lâm sàng. Tiêu chí như Lipinski, Ghose, Veber, Egan, và Muegge giúp xác định liệu hợp chất có khả năng trở thành thuốc tiềm năng không.

**ENGLISH**:
Drug-likeness analysis is a crucial step in preclinical screening. Criteria such as Lipinski, Ghose, Veber, Egan, and Muegge help assess the potential of a compound to become a drug candidate.

---

## 🔬 Mô hình phân tích / Analysis model

Chúng ta sử dụng RDKit để:

1. Chuyển đổi SMILES → Mol
2. Tính các descriptor như: MolWt, LogP, HDonors, HAcceptors, TPSA, NumRotatableBonds
3. Kiểm tra tiêu chí Lipinski và các quy tắc khác

---

## 🛠️ Hướng dẫn xử lý

### 📁 Chuẩn bị dữ liệu từ SQL (Giới hạn 100 dòng)

#### ✅ Ví dụ 1: Trích xuất compound với IC50 có giá trị số

```sql
-- T5_1_extract_drug_like_data.sql
SELECT md.chembl_id, cs.canonical_smiles, act.standard_value::float AS ic50_nm
FROM activities act
JOIN compound_structures cs ON act.molregno = cs.molregno
JOIN molecule_dictionary md ON md.molregno = act.molregno
WHERE act.standard_type = 'IC50'
  AND act.standard_value IS NOT NULL
  AND act.standard_value ~ '^[0-9\.]+$'
LIMIT 100;
```

**💡 Lưu ý:** Nếu gặp lỗi `operator does not exist: numeric ~ unknown`, hãy dùng ép kiểu:

```sql
AND act.standard_value::text ~ '^[0-9\.]+$'
```

---

## 📊 Phân tích Python

```python
# T5_2_drug_likeness_analysis.py
import pandas as pd
from rdkit import Chem
from rdkit.Chem import Descriptors, Crippen, Lipinski
import os
import warnings

# Disable user warnings
warnings.filterwarnings("ignore", category=UserWarning)

# Load data
base_path = ".."
file_path = os.path.join(base_path, "data", "T5_ic50_data.csv")
df = pd.read_csv(file_path)

# Convert SMILES to Mol
df['mol'] = df['canonical_smiles'].apply(Chem.MolFromSmiles)

# Calculate drug-likeness descriptors
df['MolWt'] = df['mol'].apply(Descriptors.MolWt)
df['LogP'] = df['mol'].apply(Crippen.MolLogP)
df['HDonors'] = df['mol'].apply(Lipinski.NumHDonors)
df['HAcceptors'] = df['mol'].apply(Lipinski.NumHAcceptors)
df['TPSA'] = df['mol'].apply(Descriptors.TPSA)
df['RotBonds'] = df['mol'].apply(Descriptors.NumRotatableBonds)

# Apply Lipinski's Rule of 5
df['Lipinski_Pass'] = (
    (df['MolWt'] <= 500) &
    (df['LogP'] <= 5) &
    (df['HDonors'] <= 5) &
    (df['HAcceptors'] <= 10)
)

df_result = df[['chembl_id', 'MolWt', 'LogP', 'HDonors', 'HAcceptors', 'TPSA', 'RotBonds', 'Lipinski_Pass']]
df_result.head()
```

---

## 📦 5 ví dụ áp dụng thực tế

| STT | Tên mục tiêu                         | SQL hoặc Python | Mô tả ngắn                        |
| --- | ------------------------------------ | --------------- | --------------------------------- |
| 1️⃣ | Lấy dữ liệu IC50 < 100 nM            | SQL             | Giới hạn 100 dòng chuẩn           |
| 2️⃣ | Tính toàn bộ descriptor RDKit        | Python          | Định lượng MolWt, LogP, TPSA,...  |
| 3️⃣ | Lọc hợp chất thỏa mãn Lipinski       | Python          | Thêm cột `Lipinski_Pass`          |
| 4️⃣ | Xuất file `T5_druglike_filtered.csv` | Python          | Chỉ lưu các hợp chất pass         |
| 5️⃣ | Vẽ histogram phân bố LogP & MolWt    | Python          | Trực quan phân bố tính chất thuốc |

---

### 📈 Ví dụ 5: Visualization (thêm)

```python
import matplotlib.pyplot as plt
import seaborn as sns

plt.figure(figsize=(10, 5))
sns.histplot(df['LogP'], bins=30, kde=True)
plt.title("Distribution of LogP")
plt.xlabel("LogP")
plt.ylabel("Count")
plt.grid(True)
plt.show()
```

---

## 📂 AIMLOps File Naming

* SQL: `../data/T5_1_extract_drug_like_data.sql`
* Notebook: `notebook/T5_1_druglikeness_exploration.ipynb`
* Python module (nếu có): `src/druglikeness.py`

---

Bạn có muốn tôi:

* Viết lại đoạn code thành file .ipynb mẫu?
* Xuất ra các file zip cấu trúc AIMLOps?
* Tạo dashboard Streamlit kiểm tra tính giống thuốc?

Hãy cho tôi biết bước tiếp theo bạn muốn thực hiện.


In [1]:
# T5_2_drug_likeness_analysis.py
import pandas as pd
from rdkit import Chem
from rdkit.Chem import Descriptors, Crippen, Lipinski
import os
import warnings

# Disable user warnings
warnings.filterwarnings("ignore", category=UserWarning)

# Load data
base_path = ".."
file_path = os.path.join(base_path, "data", "T5_ic50_data.csv")
df = pd.read_csv(file_path)

# Convert SMILES to Mol
df['mol'] = df['canonical_smiles'].apply(Chem.MolFromSmiles)

# Calculate drug-likeness descriptors
df['MolWt'] = df['mol'].apply(Descriptors.MolWt)
df['LogP'] = df['mol'].apply(Crippen.MolLogP)
df['HDonors'] = df['mol'].apply(Lipinski.NumHDonors)
df['HAcceptors'] = df['mol'].apply(Lipinski.NumHAcceptors)
df['TPSA'] = df['mol'].apply(Descriptors.TPSA)
df['RotBonds'] = df['mol'].apply(Descriptors.NumRotatableBonds)

# Apply Lipinski's Rule of 5
df['Lipinski_Pass'] = (
    (df['MolWt'] <= 500) &
    (df['LogP'] <= 5) &
    (df['HDonors'] <= 5) &
    (df['HAcceptors'] <= 10)
)

df_result = df[['chembl_id', 'MolWt', 'LogP', 'HDonors', 'HAcceptors', 'TPSA', 'RotBonds', 'Lipinski_Pass']]
df_result.head()

,chembl_id,MolWt,LogP,HDonors,HAcceptors,TPSA,RotBonds,Lipinski_Pass
0,CHEMBL308930,615.680,3.3275,0,10,125.54,10,False
1,CHEMBL308930,615.680,3.3275,0,10,125.54,10,False
2,CHEMBL292697,328.367,4.6788,1,3,50.44,3,True
3,CHEMBL289142,468.244,2.5472,3,6,127.67,7,True
4,CHEMBL289142,468.244,2.5472,3,6,127.67,7,True
